In [1]:
# 패키지 로드
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os, random

from scipy import sparse
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

from datetime import datetime
from datetime import timedelta

In [2]:
# 경로 설정
data_path = '../data'
saved_path = './saved'
output_path = './submission'

In [3]:
#bd = pd.read_csv('buy_data.csv')
#hd = pd.read_csv('history_data.csv')
mdp = pd.read_csv('meta_data_plus.csv')
md = pd.read_csv('meta_data.csv')
#prd = pd.read_csv('profile_data.csv')
#sd = pd.read_csv('search_data.csv')
#wd = pd.read_csv('watch_e_data.csv')
#ss = pd.read_csv('sample_submission.csv')

In [4]:
# 데이터 불러오기 
history = pd.read_csv(os.path.join(data_path, 'history_data.csv'), encoding='utf-8')
watch = pd.read_csv(os.path.join(data_path, 'watch_e_data.csv'), encoding='utf-8')
buy = pd.read_csv(os.path.join(data_path, 'buy_data.csv'), encoding='utf-8') 
search = pd.read_csv(os.path.join(data_path, 'search_data.csv'), encoding='utf-8')
profile = pd.read_csv(os.path.join(data_path, 'profile_data.csv'), encoding='utf-8')
meta = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')
metaplus = pd.read_csv(os.path.join(data_path, 'meta_data_plus.csv'), encoding='utf-8')

In [5]:
mt = meta.copy()
mtp = metaplus.copy()

In [6]:
print(mt.shape)
print(mtp.shape)

(42602, 16)
(767948, 4)


In [7]:
mt.head()

,album_id,title,sub_title,genre_large,genre_mid,genre_small,country,run_time,onair_date,cast_1,cast_2,cast_3,cast_4,cast_5,cast_6,cast_7
0,749,어둠이 무서워요,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
1,750,우리는 친구,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
2,2131,타요의 첫 운행,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
3,2625,길 잃은 타요,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN
4,2594,새내기 꼬마 버스의 하루,꼬마버스 타요1,키즈,TV만화,NaN,한국,660,NaN,타요,로기,라니,가니,시투,NaN,NaN


In [8]:
mtp

,album_id,keyword_type,keyword_name,keyword_value
0,749,05010101,시리즈,0
1,749,05010302,EBS,0
2,749,05010401,3세,0
3,749,05010402,4세,0
4,749,05010407,1세,0
...,...,...,...,...
767943,11629,05040104,신나는,0
767944,11629,0504010b,따라하고 싶은,0
767945,11629,05070e01,인체,0
767946,11629,05080401,캐릭터 친구들,0


In [9]:
mt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42602 entries, 0 to 42601
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   album_id     42602 non-null  int64  
 1   title        42602 non-null  object 
 2   sub_title    42602 non-null  object 
 3   genre_large  42602 non-null  object 
 4   genre_mid    42602 non-null  object 
 5   genre_small  13419 non-null  object 
 6   country      33734 non-null  object 
 7   run_time     42602 non-null  int64  
 8   onair_date   5344 non-null   float64
 9   cast_1       27603 non-null  object 
 10  cast_2       22048 non-null  object 
 11  cast_3       16463 non-null  object 
 12  cast_4       12485 non-null  object 
 13  cast_5       6382 non-null   object 
 14  cast_6       2609 non-null   object 
 15  cast_7       762 non-null    object 
dtypes: float64(1), int64(2), object(13)
memory usage: 5.2+ MB


In [10]:
mt.nunique()

album_id       39875
title          36185
sub_title       2373
genre_large        6
genre_mid         29
genre_small       20
country           20
run_time        2100
onair_date      3052
cast_1           689
cast_2           511
cast_3           396
cast_4           304
cast_5           182
cast_6            71
cast_7            21
dtype: int64

In [11]:
mtp.nunique()

album_id         39875
keyword_type      1101
keyword_name      1062
keyword_value        6
dtype: int64

In [12]:
mt.country.unique()

array(['한국', '저지', nan, '미국', '영국', '일본', '중국', '프랑스', '오스트리아', '독일',
       '이탈리아', '크로아티아', '스위스', '캐나다', '아르헨티나', '우크라이나', '핀란드', '이스라엘',
       '벨기에', '벨라루스', '네덜란드'], dtype=object)

In [13]:
mt.genre_small.unique()

array([nan, '영어', '창의학습', '수학과학', '한글', '예술교육', '코믹/명랑', '독서동화', '생활교육',
       '순정/학원', '액션/모험', '어린이/가족', '기타', '오페라', '한자', '클래식 콘서트', '장난감놀이',
       '미술/전시', 'EBS', 'SF/판타지', '체육안전'], dtype=object)

In [18]:
mt_nocast = mt.drop(['cast_1','cast_2','cast_3','cast_4','cast_5','cast_6','cast_7','genre_small'],axis = 'columns')

mt_nocast


,album_id,title,sub_title,genre_large,genre_mid,country,run_time,onair_date
0,749,어둠이 무서워요,꼬마버스 타요1,키즈,TV만화,한국,660,NaN
1,750,우리는 친구,꼬마버스 타요1,키즈,TV만화,한국,660,NaN
2,2131,타요의 첫 운행,꼬마버스 타요1,키즈,TV만화,한국,660,NaN
3,2625,길 잃은 타요,꼬마버스 타요1,키즈,TV만화,한국,660,NaN
4,2594,새내기 꼬마 버스의 하루,꼬마버스 타요1,키즈,TV만화,한국,660,NaN
...,...,...,...,...,...,...,...,...
42597,39873,로티프렌즈와 색칠놀이! - 그리피 ＆ 사탕 바구니,로티프렌즈 미술놀이,키즈,놀이교실,한국,477,NaN
42598,39874,로티프렌즈와 색칠놀이! - 베블리 ＆ 꽃,로티프렌즈 미술놀이,키즈,놀이교실,한국,466,NaN
42599,4779,손가락을 빨게 돼요,4-5세,키즈,책,한국,293,NaN
42600,4779,손가락을 빨게 돼요,아이맘콕,키즈,책,한국,293,NaN


### 같은 album_id인데 빈값이 있으면 같은 항목으로 채워주기

In [58]:
for i in mt_nocast['album_id'].value_counts().index : 
    
    if mt_nocast['album_id'].value_counts().loc[i] > 1 :
        if mt_nocast[mt_nocast['album_id']== i].country.isnull() is True : 
            print(mt_nocast[mt_nocast['album_id']== i])
    
print("done")
##같은 album_id인데 국가가 비어있는 항목은 없음

done


### meta데이터에서 Nan 값을 ‘기타’항목으로 변환

In [64]:
mt_nocast[mt_nocast['country'].isnull() == True ]

,album_id,title,sub_title,genre_large,genre_mid,country,run_time,onair_date
214,26106,도도새아저씨의 목걸이,숲 속 친구 파파룰라1,키즈,놀이교실,NaN,893,NaN
215,26107,모아의 보물창고,숲 속 친구 파파룰라1,키즈,놀이교실,NaN,872,NaN
216,26108,샤샤의 이불,숲 속 친구 파파룰라1,키즈,놀이교실,NaN,803,NaN
217,26109,파파룰라숲 이야기,숲 속 친구 파파룰라1,키즈,놀이교실,NaN,845,NaN
218,26110,봄맞이 체력단련,리얼리티쇼 유아독존,키즈,예능,NaN,1579,NaN
...,...,...,...,...,...,...,...,...
17299,24432,쉬는 시간 팝핑보바 빙수 만들기,친한친구 모여라1,키즈,예능,NaN,539,NaN
17300,23782,워터파크 체력테스트 1,친한친구 모여라1,키즈,예능,NaN,600,NaN
17301,24433,워터파크 체력테스트 2,친한친구 모여라1,키즈,예능,NaN,839,NaN
17302,24434,여름방학 생활계획표,친한친구 모여라1,키즈,예능,NaN,827,NaN


In [67]:
mt_nocast[mt_nocast['country'].isnull() == True ].replace(np.NaN, '기타')

,album_id,title,sub_title,genre_large,genre_mid,country,run_time,onair_date
214,26106,도도새아저씨의 목걸이,숲 속 친구 파파룰라1,키즈,놀이교실,기타,893,기타
215,26107,모아의 보물창고,숲 속 친구 파파룰라1,키즈,놀이교실,기타,872,기타
216,26108,샤샤의 이불,숲 속 친구 파파룰라1,키즈,놀이교실,기타,803,기타
217,26109,파파룰라숲 이야기,숲 속 친구 파파룰라1,키즈,놀이교실,기타,845,기타
218,26110,봄맞이 체력단련,리얼리티쇼 유아독존,키즈,예능,기타,1579,기타
...,...,...,...,...,...,...,...,...
17299,24432,쉬는 시간 팝핑보바 빙수 만들기,친한친구 모여라1,키즈,예능,기타,539,기타
17300,23782,워터파크 체력테스트 1,친한친구 모여라1,키즈,예능,기타,600,기타
17301,24433,워터파크 체력테스트 2,친한친구 모여라1,키즈,예능,기타,839,기타
17302,24434,여름방학 생활계획표,친한친구 모여라1,키즈,예능,기타,827,기타


In [115]:
temp = mt_nocast
temp.isnull().sum()

album_id           0
title              0
sub_title          0
genre_large        0
genre_mid          0
country         8868
run_time           0
onair_date     37258
dtype: int64

In [118]:
for i in temp[temp['country'].isnull()].index : 
    temp.loc[i,'country']= '기타'
    
    
temp.isnull().sum()

album_id           0
title              0
sub_title          0
genre_large        0
genre_mid          0
country            0
run_time           0
onair_date     37258
dtype: int64

In [119]:
temp

,album_id,title,sub_title,genre_large,genre_mid,country,run_time,onair_date
0,749,어둠이 무서워요,꼬마버스 타요1,키즈,TV만화,한국,660,NaN
1,750,우리는 친구,꼬마버스 타요1,키즈,TV만화,한국,660,NaN
2,2131,타요의 첫 운행,꼬마버스 타요1,키즈,TV만화,한국,660,NaN
3,2625,길 잃은 타요,꼬마버스 타요1,키즈,TV만화,한국,660,NaN
4,2594,새내기 꼬마 버스의 하루,꼬마버스 타요1,키즈,TV만화,한국,660,NaN
...,...,...,...,...,...,...,...,...
42597,39873,로티프렌즈와 색칠놀이! - 그리피 ＆ 사탕 바구니,로티프렌즈 미술놀이,키즈,놀이교실,한국,477,NaN
42598,39874,로티프렌즈와 색칠놀이! - 베블리 ＆ 꽃,로티프렌즈 미술놀이,키즈,놀이교실,한국,466,NaN
42599,4779,손가락을 빨게 돼요,4-5세,키즈,책,한국,293,NaN
42600,4779,손가락을 빨게 돼요,아이맘콕,키즈,책,한국,293,NaN


In [120]:
temp.to_csv("pre_meta.csv", index = False)

In [121]:
temp1 = pd.read_csv(os.path.join(data_path, 'pre_meta.csv'), encoding='utf-8')


In [124]:
temp1.isnull().sum()

album_id           0
title              0
sub_title          0
genre_large        0
genre_mid          0
country            0
run_time           0
onair_date     37258
dtype: int64